In [54]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

%matplotlib inline

LOWEST_YEAR = 1988 # This is the lowest year available in Dataset
HIGHEST_YEAR = 2016 # This is the highest year available in Dataset
START_YEAR = 2000
END_YEAR = 2017 # We're using 2017 because range() functions are exclusive for the last value

In [2]:
data = pd.read_csv('./commodity_trade_statistics_data.csv')

/Users/mihailojoksimovic/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
is_export = data['flow'] == 'Export'
is_live_animals = data['category'] == '01_live_animals'

country_summary_data = data[(data['country_or_area'] == 'USA') & is_export & is_live_animals][['year', 'trade_usd']].groupby('year').sum()

In [4]:
country_summary_data.head(5)

,trade_usd
year,
1991,692654489
1992,637981181
1993,522006551
1994,590751118
1995,544131807


In [5]:
country_summary_data.index.sort_values()[0] # First
country_summary_data.index.sort_values()[-1] # Last
country_summary_data.index.sort_values()[-2:] # Last two
country_summary_data.index.sort_values()[:-2] # All but last two

all_data = country_summary_data.sort_index()['trade_usd'].astype('float').values
train_data = all_data[:-1] # All data excluding last year
actual = all_data[-1:] # Only last year

In [6]:
model = ARMA(train_data, order=(0, 1))
model_fit = model.fit()
predicted = model_fit.predict(len(train_data), len(train_data))

In [7]:
countries = data[is_export & is_live_animals]['country_or_area'].unique()

In [14]:
# Make a dataset where index is countries names and columns will be predictions from different models

countries_df = pd.DataFrame(index=countries)

In [51]:
# Prediction algorithms

prediction_algorithms = {
    'autoregression':  AR(train_data),
    'arma': ARMA(train_data, order=(0, 1)),
    'arima': ARIMA(train_data, order=(1, 1, 1)),
    'sarimax': SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 1)),
    'simple_exponential_smoothing': SimpleExpSmoothing(train_data)
}

NameError: name 'SimpleExpSmoothing' is not defined

In [50]:
for k, model in prediction_algorithms.items():
    print("Training algorigthm: " + k)
    model_fit = model.fit()
    predicted = model_fit.predict(len(train_data), len(train_data))
    
    diff = actual - predicted
    
    print("Predicted: " + str(predicted[0]) + ", Actual: " + str(actual[0]) + ", Diff: " + str(diff[0]))


Training algorigthm: autoregression
Predicted: 740520393.074, Actual: 784223079.0, Diff: 43702685.9256
Training algorigthm: arma
Predicted: 711851161.727, Actual: 784223079.0, Diff: 72371917.2727
Training algorigthm: arima
Predicted: 31159460.1779, Actual: 784223079.0, Diff: 753063618.822
Training algorigthm: sarimax
Predicted: 783464474.794, Actual: 784223079.0, Diff: 758604.205734


/Users/mihailojoksimovic/miniconda3/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
